In [1]:
from scipy import ndimage
from skimage import io,util,color
from IPython import display
import numpy
import sys
import math
import random

import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import display, clear_output

import matplotlib.pyplot as plt

In [2]:
def readAndShowImage(image):
    im = io.imread(image)
    io.imshow(im)
    io.show()

inputImage = widgets.Dropdown(
    options={
        'Strand':'strand.jpeg',
        'Mandrill':'mandrill.png',
        'Trump':'trump.jpg',
        'Gruppe':'gruppe.jpg',
        'RGB':"rgb.png",
        'Dress':'TheDress.png'
        },
    description='Bilde: ',
    disabled=False,
)

interactive(readAndShowImage, image=inputImage)

interactive(children=(Dropdown(description='Bilde: ', options={'Strand': 'strand.jpeg', 'Mandrill': 'mandrill.…

In [3]:
def gaussian(im, sigma):
    return ndimage.gaussian_filter(im,sigma)

def nonMax(M,alpha):
    gN = numpy.zeros(M.shape)
    height, width = gN.shape
    
    convertRatio = math.pi/180
    
    for i in range(height):
        for j in range(width):
            #finn retningen D som passer best med kanten i alpha
            d = alpha[i,j]
            
            if(d < 0):
                d += 360
                
            if ((j+1) < width) and ((j-1) >= 0) and ((i+1) < height) and ((i-1) >= 0):
                # 0 degrees
                if (alpha[i,j] >= 337.5 or alpha[i,j] < 22.5) or (alpha[i,j] >= 157.5 and alpha[i,j] < 202.5):
                    if M[i,j] >= M[i,j+1] and M[i,j] >= M[i,j-1]:
                        gN[i,j] = M[i,j]
                # 45 degrees
                if (alpha[i,j] >= 22.5 and alpha[i,j] < 67.5) or (alpha[i,j] >= 202.5 and alpha[i,j] < 247.5):
                    if M[i,j] >= M[i-1,j+1] and M[i,j] >= M[i+1,j-1]:
                        gN[i,j] = M[i,j]
                # 90 degrees
                if (alpha[i,j] >= 67.5 and alpha[i,j] < 112.5) or (alpha[i,j] >= 247.5 and alpha[i,j] < 292.5):
                    if M[i,j] >= M[i-1,j] and M[i,j] >= M[i+1,j]:
                        gN[i,j] = M[i,j]
                # 135 degrees
                if (alpha[i,j] >= 112.5 and alpha[i,j] < 157.5) or (alpha[i,j] >= 292.5 and alpha[i,j] < 337.5):
                    if M[i,j] >= M[i-1,j-1] and M[i,j] >= M[i+1,j+1]:
                        gN[i,j] = M[i,j]
            
                             
                             
    return gN
    
def cannyEdge(im,sigma):
    #Finn et utjevnet gradientbilde ved  ̊a bruke førstederiverte av en Gaussian
    
    smooth = gaussian(im, sigma)
    
    sobelX, sobelY = (numpy.asarray([[-1,0,1],[-2,0,2],[-1,0,1]]),numpy.asarray([[-1,-2,-1],[0,0,0],[1,2,1]]))
    sobelX, sobelY = (ndimage.convolve(smooth,sobelX),ndimage.convolve(smooth,sobelY))
    
    M = (sobelX**2 + sobelY**2)**0.5
    alpha = numpy.arctan2(sobelY,sobelX)
    
    alpha *= 180/math.pi
    
    #Utfør nonmax suppression (ikkemax-fjerning)
    
    gN = nonMax(M,alpha)
    
    #Bruk hysteresis thresholding (dual threshold oppsett) for  ̊a finne
    
    return gN

In [6]:
def update(image,sigma):
    im = io.imread(image,as_gray=True)
    io.imshow(cannyEdge(im,sigma) > 0, cmap='gray')
    io.show()
    
interactive(update, image=inputImage, sigma=(1.0,5.0))

interactive(children=(Dropdown(description='Bilde: ', index=5, options={'Strand': 'strand.jpeg', 'Mandrill': '…